In [ ]:
# Install MALLET 2.0.8RC3 from the official site
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8RC3.zip -O ~/mallet-2.0.8RC3.zip

# Unzip MALLET
!unzip ~/mallet-2.0.8RC3.zip -d ~/
!rm ~/mallet-2.0.8RC3.zip

# Rename the folder for convenience
!mv ~/mallet-2.0.8RC3 ~/mallet-2.0.8

# Define the path to the "mallet" binary
import os
mallet_dir = os.path.expanduser('~/mallet-2.0.8')
path_to_mallet = os.path.join(mallet_dir, 'bin', 'mallet')
print(f"Path to MALLET: {path_to_mallet}")

# Optionally check Java version
!java -version

In [ ]:
!pip install little_mallet_wrapper openpyxl

In [1]:
# -*- coding: utf-8 -*-
"""
Integrated script for MALLET-based topic modeling with per-file Fisher’s Exact
filtering and stopword exclusion. UTF-8 friendly.
Outputs:
  - Topic keys
  - Per-document topic distributions
  - Excel workbook with:
      • Topic tokens
      • Token counts per document
      • Topic probabilities per document
  - Separate Excel workbook with top-document titles per topic

Graphical output (heatmap & boxplots) is disabled in this pipeline.
See plotting_helpers.py for on-demand plotting.
"""


import os
import csv
import time
from pathlib import Path
from collections import Counter

import pandas as pd
from scipy.stats import fisher_exact
import little_mallet_wrapper
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import pickle

# Change this path to point to the local MALLET binary in the current directory.
# For example, place the mallet-2.0.8 folder in the same directory as this script.
# Then use os.path.join(...) relative to the current working directory (codespace).
mallet_dir = os.path.expanduser('~/mallet-2.0.8')
path_to_mallet = os.path.join(mallet_dir, 'bin', 'mallet')

# These will be set by main() and picked up by plotting_helpers.py
OUTPUT_DIR = None
NUM_TOPICS = None

def tokenize_file(filepath):
    """Load a UTF-8 text file and split on whitespace (preserves accents)."""
    with open(filepath, "r", encoding="utf-8") as f:
        return f.read().split()

def list_txt_files(directory):
    """Return a sorted list of all .txt filenames in a directory."""
    return sorted(fn for fn in os.listdir(directory) if fn.endswith(".txt"))

def list_csv_files(directory):
    """
    Recursively find all .csv files under a directory, skipping
    any .ipynb_checkpoints folders.
    """
    csv_paths = []
    for root, dirs, files in os.walk(directory):
        if ".ipynb_checkpoints" in dirs:
            dirs.remove(".ipynb_checkpoints")
        for fn in files:
            if fn.lower().endswith(".csv"):
                csv_paths.append(os.path.join(root, fn))
    return csv_paths

def choose_directory(prompt):
    """
    Display a numbered list of immediate subdirectories (plus current dir)
    and return the full path selected by the user.
    """
    cwd = os.getcwd()
    base = Path(cwd).name
    subdirs = [
        d for d in os.listdir(cwd)
        if os.path.isdir(os.path.join(cwd, d)) and d != ".ipynb_checkpoints"
    ]
    options = [(os.path.join(cwd, d), f"{base}/{d}") for d in subdirs]
    options.append((cwd, base))

    print(prompt)
    for i, (_, label) in enumerate(options, start=1):
        print(f"{i}. {label}")

    while True:
        try:
            choice = int(input("Enter number: ").strip())
            if 1 <= choice <= len(options):
                return options[choice - 1][0]
        except ValueError:
            pass
        print("Invalid choice, please try again.")

def choose_files(filenames):
    """
    Let user pick one or more filenames by:
      - indices ("2" or "1,3")
      - index ranges ("1-4")
      - prefix matching ("report")
      - or the keyword "all" to select every file.
    Returns a sorted, unique list of chosen filenames.
    """
    print("Available files:")
    for i, fn in enumerate(filenames, start=1):
        print(f"{i}. {fn}")
    choice = input("Select files (indices, ranges, prefix, or 'all'): ").strip().lower()

    if choice == "all":
        return filenames.copy()

    selected = []
    for part in choice.split(","):
        part = part.strip()
        if part == "all":
            return filenames.copy()
        if "-" in part:
            a, b = map(int, part.split("-"))
            selected.extend(filenames[a - 1:b])
        elif part.isdigit():
            selected.append(filenames[int(part) - 1])
        else:
            selected.extend(fn for fn in filenames if fn.startswith(part))
    return sorted(set(selected))

def choose_csv_file(csv_paths):
    """
    Prompt the user to select one CSV from a list of paths.
    Returns the chosen filepath.
    """
    cwd = os.getcwd()
    base = Path(cwd).name
    print("Select your stopwords CSV:")
    for i, full in enumerate(csv_paths, start=1):
        rel = os.path.relpath(full, cwd)
        print(f"{i}. {base}/{rel}")

    while True:
        try:
            choice = int(input("Enter number: ").strip())
            if 1 <= choice <= len(csv_paths):
                return csv_paths[choice - 1]
        except ValueError:
            pass
        print("Invalid choice, please try again.")

def read_stopwords(filepath):
    """Load stopwords from a CSV, splitting on commas and trimming whitespace."""
    sw = []
    with open(filepath, "r", encoding="utf-8") as f:
        for row in csv.reader(f):
            for cell in row:
                sw.extend(cell.split(","))
    return [w.strip() for w in sw if w.strip()]

def get_fishers(word, freq_dict, rate_dict, alternative="greater"):
    """
    Perform Fisher’s Exact Test on one token:
        [[observed, total-observed],
         [expected, total-expected]]
    Returns the p-value.
    """
    observed = freq_dict.get(word, 0)
    total = sum(freq_dict.values())
    expected = round(rate_dict.get(word, 0) * total)
    table = [
        [observed, total - observed],
        [expected, total - expected]
    ]
    _, pval = fisher_exact(table, alternative=alternative)
    return pval

def calculate_rate_dictionary(rate_files, rate_dir):
    """
    Build a background rate dictionary from reference documents.
    Returns a mapping { token: relative_frequency }.
    """
    counter = Counter()
    total_tokens = 0
    for fn in rate_files:
        tokens = tokenize_file(os.path.join(rate_dir, fn))
        counter.update(tokens)
        total_tokens += len(tokens)
    return {tok: cnt / total_tokens for tok, cnt in counter.items()}

def prepare_training_data(files, directory, stopwords, rate_dict, alpha):
    """
    For each file:
      1. Tokenize and count every token.
      2. Exclude stopwords and tokens with Fisher p-value ≥ alpha.
      3. Collect filtered document text and raw token counts.
    Prints per-file progress with elapsed time.
    Returns:
      - docs: list of filtered document strings for MALLET input
      - distributions: list of Counter objects of raw token counts
    """
    docs = []
    distributions = []
    total = len(files)
    overall_start = time.time()

    for idx, fn in enumerate(files, start=1):
        file_start = time.time()
        print(f"[{idx}/{total}] ⏳ Processing '{fn}'... ")

        path = os.path.join(directory, fn)
        tokens = tokenize_file(path)
        freq = Counter(tokens)

        filtered = [
            w for w in tokens
            if w not in stopwords and get_fishers(w, freq, rate_dict) < alpha
        ]
        docs.append(" ".join(filtered))
        distributions.append(freq)

        elapsed = time.time() - file_start
        kept = len(filtered)
        before = len(tokens)
        pct = (kept / before * 100) if before else 0
        print(f"done in {elapsed:.1f}s – kept {kept}/{before} tokens ({pct:.1f}%).")

    total_elapsed = time.time() - overall_start
    print(f"[Done] Prepared {total} documents in {total_elapsed:.1f}s.\n")
    return docs, distributions

def train_topic_model(training_docs, num_topics, output_dir):
    """
    Train a MALLET model via little_mallet_wrapper
    Returns:
      - topics: list of token lists (topic keys)
      - doc_topics: list of probability lists (per-document topic distributions)
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Make sure MALLET is installed and that path_to_mallet points to the local file in the codespace.
    little_mallet_wrapper.quick_train_topic_model(
        path_to_mallet,
        output_dir,
        num_topics,
        training_docs
    )

    key_file = os.path.join(output_dir, f"mallet.topic_keys.{num_topics}")
    topics = little_mallet_wrapper.load_topic_keys(key_file)

    dist_file = os.path.join(output_dir, f"mallet.topic_distributions.{num_topics}")
    doc_topics = little_mallet_wrapper.load_topic_distributions(dist_file)

    return topics, doc_topics

def save_results_to_excel(excel_path, topics, token_distributions, doc_topics, files):
    """
    Create an Excel workbook with:
      1) 'Topics' sheet: one row per topic (Topic#, top tokens...)
      2) One sheet per document: raw token counts (removing '_stemmed' suffix)
      3) 'DocTopicDist' sheet: topic probabilities per document
    """
    wb = Workbook()
    ws0 = wb.active
    ws0.title = "Topics"
    for idx, topic in enumerate(topics):
        ws0.append([f"Topic {idx}"] + topic)

    for fn, dist in zip(files, token_distributions):
        sheet_name = Path(fn).stem.replace("_stemmed", "")
        sheet = wb.create_sheet(title=sheet_name)
        df = pd.DataFrame.from_dict(dist, orient="index", columns=["count"])
        for row in dataframe_to_rows(df, index=True, header=True):
            sheet.append(row)

    ws3 = wb.create_sheet(title="DocTopicDist")
    header = ["Document"] + [f"Topic{t}" for t in range(len(topics))]
    ws3.append(header)
    for fn, probs in zip(files, doc_topics):
        doc_name = Path(fn).stem.replace("_stemmed", "")
        ws3.append([doc_name] + [round(p, 4) for p in probs])

    wb.save(excel_path)

def save_top_titles_excel(xlsx_path, topics, training_docs, doc_topics, doc_titles, n_docs):
    """
    Save the top n_docs document titles per topic into an Excel file,
    one sheet per topic named 'Topic{#}', and remove the default blank sheet.
    """
    wb = Workbook()
    default_sheet = wb.active
    wb.remove(default_sheet)

    for t_idx in range(len(topics)):
        ws = wb.create_sheet(title=f"Topic{t_idx}")
        ws.append(["Probability", "Document Title"])
        for prob, doc in little_mallet_wrapper.get_top_docs(
            training_docs, doc_topics, t_idx, n=n_docs
        ):
            title = doc_titles.get(doc, Path(doc).stem)
            ws.append([round(prob, 4), title])

    wb.save(xlsx_path)

def input_float(prompt, min_val=None, max_val=None):
    """Prompt until the user enters a valid float (and optionally within range)."""
    while True:
        resp = input(prompt).strip()
        try:
            val = float(resp)
            if min_val is not None and val < min_val:
                print(f"Value must be at least {min_val}.")
                continue
            if max_val is not None and val > max_val:
                print(f"Value must be at most {max_val}.")
                continue
            return val
        except ValueError:
            print("Invalid input, please enter a valid number.")

def input_int(prompt, min_val=None, max_val=None):
    """Prompt until the user enters a valid integer (and optionally within range)."""
    while True:
        resp = input(prompt).strip()
        try:
            val = int(resp)
            if min_val is not None and val < min_val:
                print(f"Value must be at least {min_val}.")
                continue
            if max_val is not None and val > max_val:
                print(f"Value must be at most {max_val}.")
                continue
            return val
        except ValueError:
            print("Invalid input, please enter a valid integer.")

def main():
    global OUTPUT_DIR, NUM_TOPICS

    # Prompt user: reuse preprocessed data or run full pipeline
    reuse = input("Reuse preprocessed data? (y/n): ").strip().lower()
    if reuse == 'y':
        # Enumerate possible folders containing preprocessed data
        cwd = os.getcwd()
        # List all subdirectories in cwd that contain 'preprocessed_docs.pkl'
        candidates = []
        for d in os.listdir(cwd):
            full_path = os.path.join(cwd, d)
            if os.path.isdir(full_path):
                pkl_path = os.path.join(full_path, "preprocessed_docs.pkl")
                if os.path.exists(pkl_path):
                    candidates.append(d)
        if not candidates:
            print("No folders with preprocessed data found in this directory.")
            return
        print("Select a folder containing preprocessed data:")
        for i, d in enumerate(candidates, start=1):
            print(f"{i}. {d}")
        while True:
            try:
                choice = int(input("Enter number: ").strip())
                if 1 <= choice <= len(candidates):
                    pre_dir = candidates[choice - 1]
                    break
            except ValueError:
                pass
            print("Invalid choice, please try again.")
        OUTPUT_DIR = os.path.join(cwd, pre_dir)
        with open(os.path.join(OUTPUT_DIR, "preprocessed_docs.pkl"), "rb") as f:
            training_docs, token_distributions, target_files = pickle.load(f)
        print(f"Loaded preprocessed data from {OUTPUT_DIR}.")
    else:
        # 1) Choose and load stopwords CSV
        stop_csv = choose_csv_file(list_csv_files(os.getcwd()))
        stopwords = read_stopwords(stop_csv)

        # 2) Build background rate dictionary
        rate_dir = choose_directory("Select reference text directory:")
        rate_files = choose_files(list_txt_files(rate_dir))
        rate_dict = calculate_rate_dictionary(rate_files, rate_dir)

        # 3) Select target files
        target_dir = choose_directory("Select target text directory:")
        target_files = choose_files(list_txt_files(target_dir))

        # 4) Output folder
        while True:
            out_sub = input("Enter name for output folder: ").strip()
            if out_sub:
                break
            print("Output folder name cannot be empty.")
        OUTPUT_DIR = os.path.join(os.getcwd(), out_sub)
        Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

        # 5) Fisher’s Exact threshold
        alpha = input_float(
            "Enter Fisher’s Exact alpha threshold (e.g. 0.05): ",
            min_val=0.0, max_val=1.0
        )

        # 6) Prepare filtered training data
        training_docs, token_distributions = prepare_training_data(
            target_files, target_dir, stopwords, rate_dict, alpha
        )

        # 7) Save preprocessed data for reuse
        with open(os.path.join(OUTPUT_DIR, "preprocessed_docs.pkl"), "wb") as f:
            pickle.dump((training_docs, token_distributions, target_files), f)
        print(f"Preprocessed data saved to {OUTPUT_DIR}.")

    # Prompt for one or more numbers of topics
    topic_str = input("Enter number(s) of topics to generate (comma-separated, e.g. 10,20,30): ").strip()
    topic_nums = [int(x) for x in topic_str.split(",") if x.strip().isdigit()]
    if not topic_nums:
        print("No valid topic numbers entered. Exiting.")
        return

    # Prompt for number of top documents per topic
    n_top_docs = input_int("Enter number of top documents per topic: ", min_val=1)
    for num_topics in topic_nums:
        print(f"\n--- Training topic model with {num_topics} topics ---")
        NUM_TOPICS = num_topics
        run_dir = os.path.join(OUTPUT_DIR, f"topics_{num_topics}")
        Path(run_dir).mkdir(parents=True, exist_ok=True)
        topics, doc_topics = train_topic_model(training_docs, NUM_TOPICS, run_dir)
        fn_list_path = os.path.join(run_dir, "input_filenames.txt")
        with open(fn_list_path, "w", encoding="utf-8") as f:
            for fn in target_files:
                f.write(fn + "\n")
        excel_results = os.path.join(run_dir, "topic_model_results.xlsx")
        save_results_to_excel(excel_results, topics, token_distributions, doc_topics, target_files)
        doc_titles = {doc: Path(fn).stem for doc, fn in zip(training_docs, target_files)}
        top_titles_path = os.path.join(run_dir, "top_titles.xlsx")
        save_top_titles_excel(
            top_titles_path, topics, training_docs, doc_topics, doc_titles, n_top_docs
        )
        print(f"✅ Finished {num_topics} topics. Results in {run_dir}.")
    print("\nAll requested topic models completed!")
    print("To create heatmaps or boxplots, import and run plotting_helpers.generate_heatmap() or generate_boxplots().")

if __name__ == "__main__":
    main()

Select your stopwords CSV:
1. lemmatized/stop_words.csv
Select reference text directory:
1. lemmatized/July1025
2. lemmatized
Available files:
1. Bodin_stemmed.txt
2. L'Hospital_stemmed.txt
Select reference text directory:
1. lemmatized/July1025
2. lemmatized
Available files:
1. Bodin_stemmed.txt
2. L'Hospital_stemmed.txt
Select target text directory:
1. lemmatized/July1025
2. lemmatized
Select target text directory:
1. lemmatized/July1025
2. lemmatized
Available files:
1. Bodin_stemmed.txt
2. L'Hospital_stemmed.txt
Available files:
1. Bodin_stemmed.txt
2. L'Hospital_stemmed.txt
[1/2] ⏳ Processing 'Bodin_stemmed.txt'... 
[1/2] ⏳ Processing 'Bodin_stemmed.txt'... 
done in 324.6s – kept 29974/753572 tokens (4.0%).
[2/2] ⏳ Processing 'L'Hospital_stemmed.txt'... 
done in 324.6s – kept 29974/753572 tokens (4.0%).
[2/2] ⏳ Processing 'L'Hospital_stemmed.txt'... 
done in 36.3s – kept 46971/263789 tokens (17.8%).
[Done] Prepared 2 documents in 360.8s.

Preprocessed data saved to /workspaces/Jer

Mallet LDA: 8 topics, 3 topic bits, 111 topic mask
Data loaded.
max tokens: 46461
total tokens: 75117
<10> LL/token: -6.39836
<20> LL/token: -6.25249
<30> LL/token: -6.16312
<40> LL/token: -6.08976

0	0.625	justic bien caus temp bon don homm roy gard vertu charl royaum lettr comt ordre jean honneur estat gen baill 
1	0.625	fort chos fin republ animal grec corp fois naturel eau souverainet voit magistrat yeux best diabl feu part proport puissanc 
2	0.625	estat bon duc procès homm droit injustic person rend mal traict offic sort franc argent laiss charg ruin affair réform 
3	0.625	natur republ fer matier fort aristocrat feu confess corp sorci malin proc rout void element esclav general dif attendu lun 
4	0.625	roy regard don comt bien honneur jeun sieur teneu compaign servyc adviz désordr richess occas just royn plais miser rapport 
5	0.625	chos part puissanc corp ame form magistrat sorci fort soleil fois republ senat aristot eau naturel air pierr populair mour 
6	0.625	roy offici consei

Complete
✅ Finished 8 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_8.

--- Training topic model with 9 topics ---
Importing data...
✅ Finished 8 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_8.

--- Training topic model with 9 topics ---
Importing data...
Complete
Training topic model...
Complete
Training topic model...


Mallet LDA: 9 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 46461
total tokens: 75117
<10> LL/token: -6.29387
<20> LL/token: -6.15148
<30> LL/token: -6.07243
<40> LL/token: -6.01965
<10> LL/token: -6.29387
<20> LL/token: -6.15148
<30> LL/token: -6.07243
<40> LL/token: -6.01965

0	0.55556	fort chos republ corp part natur fin puissanc form eau naturel fer magistrat sorci fois animal ame soleil senat aristot 
1	0.55556	feussent vent remettr saison viol tesmoing advocat voisin maig scayt charit dispos consomm recebvoir debt capitul namur oster proteg poictou 
2	0.55556	jug procès bon paix court réform roy don puiss églis vérit gouvern mérit laiss provinc priv trent violenc gaig jour 
3	0.55556	roy bien justic réform raison ordon public vertu franc jug jour estat droit don long person party temp offic ruin 
4	0.55556	bon justic bien roy comt nom honneur faveur conseil estim royaum gard jeun praticqu compt veneu assembl heureux teneu francois 
5	0.55556	chos populair feu mie

Complete
✅ Finished 9 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_9.

--- Training topic model with 10 topics ---
Importing data...
✅ Finished 9 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_9.

--- Training topic model with 10 topics ---
Importing data...
Complete
Training topic model...
Complete
Training topic model...


...

Complete
✅ Finished 11 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_11.

--- Training topic model with 12 topics ---
Importing data...
✅ Finished 11 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_11.

--- Training topic model with 12 topics ---
Importing data...
Complete
Training topic model...
Complete
Training topic model...


Mallet LDA: 12 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 46461
total tokens: 75117
<10> LL/token: -6.63093
<20> LL/token: -6.41801
<30> LL/token: -6.30809
<40> LL/token: -6.2316
<10> LL/token: -6.63093
<20> LL/token: -6.41801
<30> LL/token: -6.30809
<40> LL/token: -6.2316

0	0.41667	bon bien raison homm procès court justic jug sort party injustic don conseil gard bel lettr duc avaric gouvern offic 
1	0.41667	rend réform roy conseil vray long jug mal faveur offici nécessair chambr compaign mauvais caus outraig public justic volont obeyss 
2	0.41667	roy sainct argent comt bien honneur franc religion offic traict vertu ordre demand justic provinc francois pun profict gen remonstr 
3	0.41667	bien réform honneur person jug regard royaum estat églis couron empesch droit dignit content sieur viv plainct corrupt personnaig duc 
4	0.41667	corp naturel fer senat aristot pierr chos matier general soleil form grec proport cost fort auf avantag chaff part perfon 
5	0.41667	roy 

Complete
✅ Finished 12 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_12.

--- Training topic model with 13 topics ---
Importing data...
✅ Finished 12 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_12.

--- Training topic model with 13 topics ---
Importing data...
Complete
Training topic model...
Complete
Training topic model...


Mallet LDA: 13 topics, 4 topic bits, 1111 topic mask
Data loaded.
max tokens: 46461
total tokens: 75117
<10> LL/token: -6.63457
<20> LL/token: -6.42462
<30> LL/token: -6.31767
<10> LL/token: -6.63457
<20> LL/token: -6.42462
<30> LL/token: -6.31767
<40> LL/token: -6.23171

0	0.38462	franc estat bien party offic jug paix sort réform lettr caus demeur laiss baill sag don offici viv bon philipp 
1	0.38462	justic person duc ordon droit chanceli guer procès polic rich violenc conseil merveil administr honor nouvel mesch compt expérient appartient 
2	0.38462	jug justic royaum bon bien charg vertu vray regard rend sainct conseil religion charl mal author duch esper regn personnaig 
3	0.38462	sathan cóme prof chos auant populair venus naturel 
4	0.38462	form mieux part soleil fort ame yeux general mour matier sais aristot rout colleg natur demon feu princip auf espaigne 
5	0.38462	roy estat temp gen bien injustic argent justic général pauvr raison estim duc devoir mérit priv caus envoi crainct 

Complete
✅ Finished 13 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_13.

All requested topic models completed!
To create heatmaps or boxplots, import and run plotting_helpers.generate_heatmap() or generate_boxplots().
✅ Finished 13 topics. Results in /workspaces/Jerusalimiec-Dissertation/Text/Corpora/Concatenated/lemmatized/July1325/topics_13.

All requested topic models completed!
To create heatmaps or boxplots, import and run plotting_helpers.generate_heatmap() or generate_boxplots().


In [ ]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import little_mallet_wrapper

# Defaults for plot dimensions and output
DEFAULT_DPI = 300
DEFAULT_DIM_INCHES = (13, 9)  # (width in inches, height in inches)
OUTPUT_DIR = None  # Will be set by user input
NUM_TOPICS = None  # Will be set by user input

# Helper function to load model results from the output directory
# Loads topic keys, document-topic distributions, and filenames used in modeling
# Returns: (list of filenames, list of topics, list of doc-topic distributions)
def _load_model_results():
    """
    Load topic keys, distributions, and filenames from the last pipeline run.
    """
    if not OUTPUT_DIR or not NUM_TOPICS:
        raise RuntimeError(
            "OUTPUT_DIR and NUM_TOPICS must be set at module level. "
            "Define them before calling plotting functions."
        )

    key_file = os.path.join(OUTPUT_DIR, f"mallet.topic_keys.{NUM_TOPICS}")
    dist_file = os.path.join(OUTPUT_DIR, f"mallet.topic_distributions.{NUM_TOPICS}")
    topics = little_mallet_wrapper.load_topic_keys(key_file)
    doc_topics = little_mallet_wrapper.load_topic_distributions(dist_file)

    fn_list = os.path.join(OUTPUT_DIR, "input_filenames.txt")
    with open(fn_list, "r", encoding="utf-8") as f:
        files = [line.strip() for line in f]

    return files, topics, doc_topics

# Generate and save a heatmap of topic-by-document probabilities
# Prompts user for width, calculates figure size, and saves as PDF
# Uses little_mallet_wrapper's plotting function
# Only works if OUTPUT_DIR and NUM_TOPICS are set and model results exist
def generate_heatmap():
    """
    Prompt for a desired chart width in pixels, then generate and save a
    topic-by-document heatmap. Exports only PDF + PNG.
    """
    try:
        width_px = int(input("Enter heatmap width in pixels (e.g. 1600): ").strip())
    except ValueError:
        width_px = int(DEFAULT_DIM_INCHES[0] * DEFAULT_DPI)
        print(f"Invalid input, defaulting to {width_px} px width ({DEFAULT_DPI} DPI).")

    dpi_value = DEFAULT_DPI
    aspect = DEFAULT_DIM_INCHES[1] / DEFAULT_DIM_INCHES[0]
    height_px = int(width_px * aspect)
    figsize = (width_px / dpi_value, height_px / dpi_value)

    files, topics, doc_topics = _load_model_results()
    labels = [Path(fn).stem for fn in files]

    plt.close('all')
    pdf_out = os.path.join(OUTPUT_DIR, "categories_by_topics.pdf")
    fig = little_mallet_wrapper.plot_categories_by_topics_heatmap(
        labels,
        doc_topics,
        topics,
        pdf_out,
        target_labels=labels,
        dim=figsize
    )

    if fig is None:
        fig = plt.gcf()

    fig.canvas.draw()
    plt.close(fig)

    print(f"Heatmap saved: {pdf_out}")

# Generate and save a grid of boxplots (one subplot per topic)
# Prompts user for width, calculates figure size, and saves as JPG
# Each subplot shows the distribution of topic probabilities across documents
def generate_boxplot_grid():
    """
    Prompt for a desired chart width in pixels, then draw a grid of boxplots
    (one subplot per topic) and save as a single PDF + PNG.
    """
    try:
        width_px = int(input("Enter boxplot-grid width in pixels (e.g. 1600): ").strip())
    except ValueError:
        width_px = int(DEFAULT_DIM_INCHES[0] * DEFAULT_DPI)
        print(f"Invalid input, defaulting to {width_px} px width ({DEFAULT_DPI} DPI).")

    dpi_value = DEFAULT_DPI
    aspect = DEFAULT_DIM_INCHES[1] / DEFAULT_DIM_INCHES[0]
    height_px = int(width_px * aspect)
    figsize = (width_px / dpi_value, height_px / dpi_value)

    files, topics, doc_topics = _load_model_results()
    labels = [Path(fn).stem for fn in files]
    n_topics = len(topics)

    cols = 2
    rows = (n_topics + cols - 1) // cols

    plt.close('all')
    fig, axes = plt.subplots(rows, cols,
                             figsize=figsize,
                             dpi=dpi_value,
                             sharex=True)
    axes = axes.flatten()

    for t_idx in range(n_topics):
        ax = axes[t_idx]
        data = [doc_topics[i][t_idx] for i in range(len(doc_topics))]
        ax.boxplot(data, vert=False)
        ax.set_title(f"Topic {t_idx}")
        ax.set_yticks([])

    for ax in axes[n_topics:]:
        ax.axis("off")

    fig.tight_layout()

    png_path = os.path.join(OUTPUT_DIR, "all_topic_boxplots.jpg")
    fig.savefig(png_path, dpi=dpi_value)
    plt.close(fig)

    print(f"Boxplot grid saved: {png_path}")

# Generate and save one boxplot per topic using little_mallet_wrapper's function
# Prompts user for width, calculates figure size, and saves each as a PDF
# Each plot shows the distribution of a single topic across all documents
def generate_lmw_boxplots():
    """
    Prompt for a desired chart width in pixels, then generate and save
    boxplots for each topic using lmw's plot_categories_by_topic_boxplots.
    """
    try:
        width_px = int(input("Enter LMW boxplot width in pixels (e.g. 1600): ").strip())
    except ValueError:
        width_px = int(DEFAULT_DIM_INCHES[0] * DEFAULT_DPI)
        print(f"Invalid input, defaulting to {width_px} px width ({DEFAULT_DPI} DPI).")

    dpi_value = DEFAULT_DPI
    aspect = DEFAULT_DIM_INCHES[1] / DEFAULT_DIM_INCHES[0]
    height_px = int(width_px * aspect)
    figsize = (width_px / dpi_value, height_px / dpi_value)

    files, topics, doc_topics = _load_model_results()
    labels = [Path(fn).stem for fn in files]
    n_topics = len(topics)

    plt.close('all')
    out_paths = []
    for topic_idx in range(n_topics):
        out_path = os.path.join(
            OUTPUT_DIR, f"lmw_topic_boxplots_topic_{topic_idx}.pdf"
        )
        fig = little_mallet_wrapper.plot_categories_by_topic_boxplots(
            labels,
            doc_topics,
            topics,
            topic_idx,  # <--- This is the missing argument!
            output_path=out_path,
            target_labels=None,
            dim=figsize
        )
        if fig is None:
            fig = plt.gcf()
        fig.canvas.draw()
        plt.close(fig)
        out_paths.append(out_path)

    print(f"LMW boxplots saved: {', '.join(out_paths)}")
    
# Main entry point for plotting: prompts user for output folder, number of topics, and which plots to generate
def main():
    global OUTPUT_DIR, NUM_TOPICS

    # Prompt user for output directory and number of topics
    output_dir_name = input("Enter the output folder name: ").strip()
    OUTPUT_DIR = os.path.abspath(output_dir_name)
    num_topics_str = input("Enter the number of topics: ").strip()
    NUM_TOPICS = int(num_topics_str)

    # Prompt user to select which plots to generate
    print("Select which plots to generate:")
    print("1) Heatmap (PDF)")
    print("2) Boxplot grid (JPG)")
    print("3) LMW boxplots (PDF, label-based)")
    print("Type any combination of these numbers separated by commas (e.g. 1,2,3), or 'all' for all:")
    choice = input("Your choice: ").lower()

    if "all" in choice:
        generate_heatmap()
        generate_boxplot_grid()
        generate_lmw_boxplots()
    else:
        if '1' in choice:
            generate_heatmap()
        if '2' in choice:
            generate_boxplot_grid()
        if '3' in choice:
            generate_lmw_boxplots()

if __name__ == "__main__":
    main()

In [ ]:
import os
from pathlib import Path

# Prompt the user to select a MALLET run folder (subdirectory containing 'input_filenames.txt').
# Returns the full path to the selected folder.
def choose_mallet_run(prompt="Select a MALLET run folder that contains 'input_filenames.txt' and distributions files:"):
    """
    Let the user pick a subfolder containing 'input_filenames.txt'.
    Returns the full path to that folder.
    """
    cwd = os.getcwd()
    # List all subdirectories except .ipynb_checkpoints
    subdirs = [
        d for d in os.listdir(cwd)
        if os.path.isdir(os.path.join(cwd, d)) and d != ".ipynb_checkpoints"
    ]
    if not subdirs:
        print("No subfolders found.")
        return cwd

    print(prompt)
    for i, fn in enumerate(subdirs, start=1):
        print(f"[{i}] {fn}")
    # Prompt until a valid selection is made
    while True:
        try:
            choice = int(input("Enter number: ").strip())
            if 1 <= choice <= len(subdirs):
                return os.path.join(cwd, subdirs[choice - 1])
        except ValueError:
            pass
        print("Invalid choice, please try again.")

# Load the list of filenames (in modeling order) and the topic distributions
# for a given MALLET run and number of topics.
def load_model_data(run_dir, num_topics):
    """
    Load the file list (input_filenames.txt) and the doc-topic distributions
    for the specified number of topics from 'run_dir'.
    Ensures that the order of files matches how they were used in modeling.
    """
    fn_list_path = os.path.join(run_dir, "input_filenames.txt")
    if not os.path.exists(fn_list_path):
        raise FileNotFoundError(f"'input_filenames.txt' not found in {run_dir}.")

    # Read the filenames in the order used for modeling
    with open(fn_list_path, "r", encoding="utf-8") as f:
        files = [line.strip() for line in f]

    dist_file = os.path.join(run_dir, f"mallet.topic_distributions.{num_topics}")
    if not os.path.exists(dist_file):
        raise FileNotFoundError(f"No doc-topic distribution file for {num_topics} topics in {run_dir}.")

    import little_mallet_wrapper
    # Load the topic distributions for each file
    doc_topics = little_mallet_wrapper.load_topic_distributions(dist_file)
    return files, doc_topics

# Prompt the user to select files by index, range, or 'all'.
# Returns a sorted list of selected indices (0-based).
def select_indices_in_files(file_list, group_name):
    """
    Prompt the user to pick files by:
      - 'all'
      - single indices (e.g. '3')
      - index ranges (e.g. '2-5')
      or any combination separated by commas (e.g. '1,2-4,7').

    Returns a sorted list of chosen indices (0-based).
    """
    print(f"\nSelect indices for {group_name}. Possible choices:")
    for i, f in enumerate(file_list, start=1):
        print(f"[{i}] {f}")
    user_input = input(
        f"Enter comma-separated indices or ranges, or 'all' for {group_name}: "
    ).strip().lower()

    if user_input == "all":
        return list(range(len(file_list)))

    chosen = []
    for part in user_input.split(","):
        part = part.strip()
        if "-" in part:
            # Handle range syntax "start-end"
            try:
                start_str, end_str = part.split("-")
                start = int(start_str.strip())
                end = int(end_str.strip())
                if start <= end:
                    chosen.extend(range(start - 1, end))
                else:
                    # If user reversed them accidentally, handle gracefully:
                    chosen.extend(range(end - 1, start))
            except ValueError:
                print(f"Ignoring invalid range: '{part}'")
        else:
            # Handle single numeric index
            try:
                idx = int(part) - 1
                chosen.append(idx)
            except ValueError:
                print(f"Ignoring invalid entry: '{part}'")

    # Remove duplicates and ensure ascending order
    final_indices = sorted(set(i for i in chosen if 0 <= i < len(file_list)))
    return final_indices

# Compute the average topic distance (L1 norm) between two topic distributions.
def average_topic_distance(dist_a, dist_b):
    """
    Calculate the average distance between two topic distributions
    by summing absolute differences and dividing by the number of topics.
    """
    return sum(abs(a - b) for a, b in zip(dist_a, dist_b)) / len(dist_a)

# Main logic for comparing two groups of files:
# 1. Prompt user to select two groups of files.
# 2. Prompt user to select one file from Group 1.
# 3. Compare that file to all files in Group 2 and rank by similarity.
def compare_two_groups(files, doc_topics):
    """
    1) Divide files into two groups, picking indices from 'files'.
    2) Select one file from Group 1.
    3) Compare to every file in Group 2 and rank by average topic distance.
    """
    # 1) Split into two groups
    group1_indices = select_indices_in_files(files, "Group 1")
    group2_indices = select_indices_in_files(files, "Group 2")

    # 2) Pick one file from Group 1
    print("\nWhich file from Group 1 would you like to compare?")
    for i, idx in enumerate(group1_indices, start=1):
        print(f"[{i}] {files[idx]}")
    choice = int(input("Enter the number: ")) - 1
    chosen_idx = group1_indices[choice]

    chosen_dist = doc_topics[chosen_idx]
    print(f"\nSelected file from Group 1: {files[chosen_idx]}")

    # 3) For each file in Group 2, compute average topic distance, then sort
    distances = []
    for idx in group2_indices:
        dist = average_topic_distance(chosen_dist, doc_topics[idx])
        distances.append((files[idx], dist))

    # Sort by ascending distance (most similar = smallest distance)
    distances.sort(key=lambda x: x[1])

    print("\nRanking by similarity (ascending distance):")
    for rank, (fname, distval) in enumerate(distances, start=1):
        print(f"{rank}. {fname} (distance={distval:.4f})")

# Entry point for the comparison workflow:
# 1. Prompt user to select a MALLET run folder and number of topics.
# 2. Load the files and topic distributions.
# 3. Launch the two-group comparison.
def start_comparison():
    """
    Guide the user to pick a MALLET run folder, specify the number of topics,
    then load files and doc_topic distributions and compare two groups.
    """
    run_dir = choose_mallet_run()
    num_topics = int(input("Enter the number of topics used for that run: ").strip())
    files, doc_topics = load_model_data(run_dir, num_topics)
    compare_two_groups(files, doc_topics)

# Run the comparison workflow when this cell is executed
start_comparison()